# Setup

In [ ]:
import os 
import pandas as pd

project_id="upbeat-legacy-282508" # need to specify accordingly
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
def run_query(query):
    df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})
    print(len(df), 'rows')
    print(df.head())
    return df

# an example
df = run_query('''
SELECT
  subject_id,
  hadm_id,
  stay_id
FROM
  `physionet-data.mimic_icu.icustays`
LIMIT 10
''')
df

# Cohort extraction

Here procedure `itemid=225792` is used to identify invasive mechanical ventilation. First MV at each stay with duration longer than 24hr is used.

In [ ]:
sql_vent_cohort = '''
with tmp as (
  select stay_id, starttime, endtime, itemid 
  , DATETIME_DIFF(endtime, starttime, MINUTE) as duration
  ,ROW_NUMBER() OVER (partition by stay_id order by starttime) as firstRow
  from `physionet-data.mimic_icu.procedureevents` 
  where itemid = 225792
) 
select
stay_id, starttime, endtime, duration
,CASE
      WHEN duration > 72*60 THEN 1
    ELSE
    0
  END
    AS over72h
from tmp
where duration >= 24 * 60 and firstRow=1
order by stay_id
'''

cohort_df = run_query(sql_vent_cohort)
print('num of first ventilation cases in an ICU with duration longer than 24h', len(cohort_df))

We now need to create a view in bq for the cohort in order to continue with feature extraction.

In [ ]:
dataset_id = 'default' # change accordingly
view_id = f"{project_id}.{dataset_id}.vent_cohort_mimiciv_view"

from google.cloud import bigquery

client = bigquery.Client()

view = bigquery.Table(view_id)
view.view_query = sql_vent_cohort

view = client.create_table(view)
print('Created view')

# Feature extraction

In [ ]:
sql_height = '''
SELECT v.stay_id, he.height #v.starttime, v.endtime, he.* #, we.* 

FROM `{}` v
left join `physionet-data.mimic_derived.height` he on v.stay_id  = he.stay_id  
where he.height is not null
order by he.height 

'''.format(view_id)
height = run_query(sql_height)

In [ ]:
sql_weight = '''
with tmp as (
  SELECT v.stay_id, v.starttime, v.endtime,  max(we.starttime) as weight_time 
  FROM `{}` v
  -- left join `physionet-data.mimic_derived.height` he on v.stay_id  = he.stay_id  
  left join `physionet-data.mimic_derived.weight_durations` we on v.stay_id  = we.stay_id 
  where we.weight is not null # there are some missing values
  group by v.stay_id, v.starttime, v.endtime
  
) select t.stay_id, we.weight  from tmp t
left join `physionet-data.mimic_derived.weight_durations` we on t.stay_id  = we.stay_id and t.weight_time = we.starttime

'''.format(view_id)
weight = run_query(sql_weight)

In [ ]:
sql_demographics='''
with tmp as (
  select v.stay_id , v.starttime , a.admission_location , a.insurance , a.language ,a.ethnicity ,a.marital_status, a.admittime ,ag.age, p.gender 
  FROM `{}` v
  left join `physionet-data.mimic_icu.icustays` i on v.stay_id = i.stay_id 
  left join `physionet-data.mimic_core.admissions` a on i.subject_id = a.subject_id and i.hadm_id = a.hadm_id  
  left join `physionet-data.mimic_core.patients` p on i.subject_id =p.subject_id 
  left join `physionet-data.mimic_derived.age` ag on i.hadm_id = ag.hadm_id 
)

select 
stay_id ,admission_location ,insurance ,language ,ethnicity ,marital_status, gender, age
, DATETIME_DIFF(starttime, admittime, HOUR) as hours_in_hosp_before_intubation
FROM tmp
'''.format(view_id)
demo = run_query(sql_demographics)

In [ ]:
sql_bg='''
with tmp as (
select v.stay_id ,v.starttime ,v.endtime , bg.charttime ,bg.totalco2 ,bg.lactate, bg.ph
FROM `{}` v
left join `physionet-data.mimic_icu.icustays` i on v.stay_id = i.stay_id 
left join `physionet-data.mimic_derived.bg` bg on i.subject_id = bg.subject_id and i.hadm_id = bg.hadm_id 
where (bg.charttime >= v.starttime ) and (bg.charttime <= v.endtime ) 
)
select stay_id, 
max(totalco2) as co2_total_max,
min(totalco2) as co2_total_min,
max(ph) as ph_max,
min(ph) as ph_min,
max(lactate) as lactate_max,
min(lactate) as lactate_min,
from tmp
group by stay_id ,starttime , endtime
'''.format(view_id)
bg = run_query(sql_bg)

In [ ]:
sql_pfratio='''
with tmp as (
select v.stay_id ,v.starttime ,v.endtime ,bg.po2 ,bg.fio2 , bg.pao2fio2ratio, bg.charttime 
,ROW_NUMBER() OVER (partition by v.stay_id, v.starttime,v.endtime  order by bg.charttime DESC) as lastRow
FROM `{}` v
left join `physionet-data.mimic_icu.icustays` i on v.stay_id = i.stay_id 
left join `physionet-data.mimic_derived.bg` bg on i.subject_id = bg.subject_id and i.hadm_id = bg.hadm_id 
where (bg.charttime >= v.starttime ) and (bg.charttime <= v.endtime ) 
)
select stay_id, pao2fio2ratio, #lastRow #, starttime, endtime
from tmp 
where lastRow=1
'''.format(view_id)
pf = run_query(sql_pfratio)

In [ ]:
sql_vital = '''
select v.stay_id

, MAX(case when itemid in (220045) and valuenum > 0 and valuenum < 300 then valuenum else null end) as heart_rate_max
, MIN(case when itemid in (220045) and valuenum > 0 and valuenum < 300 then valuenum else null end) as heart_rate_min

, MAX(case when itemid = 220181 and valuenum > 0 and valuenum < 300 then valuenum else null end) as mbp_ni_max
, MIN(case when itemid = 220181 and valuenum > 0 and valuenum < 300 then valuenum else null end) as mbp_ni_min

, MAX(case when itemid in (220052) and valuenum > 0 and valuenum < 300 then valuenum else null end) as mbp_arterial_max
, MIN(case when itemid in (220052) and valuenum > 0 and valuenum < 300 then valuenum else null end) as mbp_arterial_min

, MAX(case when itemid in (220210,224690) and valuenum > 0 and valuenum < 70 then valuenum else null end) as resp_rate_max
, MIN(case when itemid in (220210,224690) and valuenum > 0 and valuenum < 70 then valuenum else null end) as resp_rate_min

, MAX(case when itemid in (220277) and valuenum > 0 and valuenum <= 100 then valuenum else null end) as spo2_max
, MIN(case when itemid in (220277) and valuenum > 0 and valuenum <= 100 then valuenum else null end) as spo2_min

, ROUND(
    MAX(case when itemid in (223761) and valuenum > 70 and valuenum < 120 then (valuenum-32)/1.8 -- converted to degC in valuenum call
             when itemid in (223762) and valuenum > 10 and valuenum < 50  then valuenum else null end)
    , 2) as temp_max
, ROUND(
    MIN(case when itemid in (223761) and valuenum > 70 and valuenum < 120 then (valuenum-32)/1.8 -- converted to degC in valuenum call
             when itemid in (223762) and valuenum > 10 and valuenum < 50  then valuenum else null end)
    , 2) as temp_min

, MAX(case when ce.itemid in (225664,220621,226537) and ce.valuenum > 0 then ce.valuenum else null end) as glucose_max
, MIN(case when ce.itemid in (225664,220621,226537) and ce.valuenum > 0 then ce.valuenum else null end) as glucose_min
FROM `{}` v
-- left join `physionet-data.mimic_icu.icustays` i on v.stay_id = i.stay_id
left join `physionet-data.mimic_icu.chartevents` ce on v.stay_id  = ce.stay_id 
where (ce.charttime >= v.starttime ) and (ce.charttime <= v.endtime ) 
 and ce.itemid in
  (
    220045, -- Heart Rate
    220181, -- Non Invasive Blood Pressure mean
    220052, -- Arterial Blood Pressure mean
    220210, -- Respiratory Rate
    224690, -- Respiratory Rate (Total)
    220277, -- SPO2, peripheral
    -- TEMPERATURE
    223762, -- "Temperature Celsius"
    223761,  -- "Temperature Fahrenheit"
    -- GLUCOSE
    225664, -- Glucose finger stick
    220621, -- Glucose (serum)
    226537 -- Glucose (whole blood)
)
group by v.stay_id
'''.format(view_id)
vitals = run_query(sql_vital)

In [ ]:
sql_med1='''
with tmp as (
  select v.stay_id
  , case when (e.starttime >= v.starttime ) and (e.endtime <= v.endtime) then 1 else 0 end as eval
  , case when (va.starttime >= v.starttime ) and (va.endtime <= v.endtime) then 1 else 0 end as vval
  , case when (db.starttime >= v.starttime ) and (db.endtime <= v.endtime) then 1 else 0 end as dbval
  FROM `{}` v
  
  left join `physionet-data.mimic_derived.epinephrine` e on v.stay_id = e.stay_id
  left join `physionet-data.mimic_derived.vasopressin` va on v.stay_id = va.stay_id
  left join `physionet-data.mimic_derived.dobutamine` db on v.stay_id = db.stay_id  
)
select stay_id 
-- , count(eval) as s
,case when count(eval)>0 then 1 else 0 end as epinephrine
,case when count(vval)>0 then 1 else 0 end as vasopressin
,case when count(dbval)>0 then 1 else 0 end as dobutamine
from tmp
group by stay_id
'''.format(view_id)
med1 = run_query(sql_med1)

In [ ]:
sql_med2='''
with tmp as (
  select v.stay_id
  , case when (n.starttime >= v.starttime ) and (n.endtime <= v.endtime) then 1 else 0 end as nval
  , case when (p.starttime >= v.starttime ) and (p.endtime <= v.endtime) then 1 else 0 end as pval
  , case when (dp.starttime >= v.starttime ) and (dp.endtime <= v.endtime) then 1 else 0 end as dpval
  FROM `{}` v
  
  left join `physionet-data.mimic_derived.norepinephrine` n on v.stay_id = n.stay_id
  left join `physionet-data.mimic_derived.phenylephrine` p on v.stay_id = p.stay_id
  left join `physionet-data.mimic_derived.dopamine` dp on v.stay_id = dp.stay_id
)
select stay_id 
,case when count(nval)>0 then 1 else 0 end as norepinephrine
,case when count(pval)>0 then 1 else 0 end as phenylephrine
,case when count(dpval)>0 then 1 else 0 end as dopamine
from tmp
group by stay_id
'''.format(view_id)
med2 = run_query(sql_med2)

In [ ]:
sql_neuroblock='''
with tmp as (
select 
v.stay_id,
case when (n.starttime >= v.starttime ) and (n.starttime <= v.endtime ) then 1 else 0 end as nb
FROM `{}` v
LEFT JOIN `physionet-data.mimic_derived.neuroblock` n on v.stay_id = n.stay_id 
)
select stay_id, case when sum(nb)>0 then 1 else 0 end as neuroblocker
FROM tmp
GROUP BY stay_id 
'''.format(view_id)
neuroblock = run_query(sql_neuroblock)

In [ ]:
sql_rrt='''
with tmp as (
  select 
  v.stay_id,
  case when (n.charttime >= v.starttime ) and (n.charttime <= v.endtime ) and (n.dialysis_present=1) then 1 else 0 end as dia
  FROM `{}` v
  LEFT JOIN `physionet-data.mimic_derived.rrt` n on v.stay_id = n.stay_id 
)
select stay_id, case when sum(dia)>0 then 1 else 0 end as rrt
FROM tmp
GROUP BY stay_id 
'''.format(view_id)
rrt = run_query(sql_rrt)

In [ ]:
sql_vent_setting='''
with tmp as (SELECT 
vc.stay_id, vc.starttime ,vc.endtime , vs.charttime ,vs.fio2 , vs.peep ,vs.plateau_pressure 
FROM `{}` vc
left join `physionet-data.mimic_icu.icustays` i on vc.stay_id = i.stay_id 
left join `physionet-data.mimic_derived.ventilator_setting` vs on i.subject_id = vs.subject_id 
where (vs.charttime >= vc.starttime ) and (vs.charttime <= vc.endtime )
)
select stay_id,
max(fio2) as fio2_max,
min(fio2) as fio2_min,
max(peep) as peep_max,
min(peep) as peep_min,
max(plateau_pressure) as plateau_pressure_max,
min(plateau_pressure) as plateau_pressure_min,

from tmp
group by stay_id
'''.format(view_id)
vent = run_query(sql_vent_setting)

In [ ]:
sql_rhythm='''
with tmp as (
select v.stay_id , r.charttime ,r.heart_rhythm , ROW_NUMBER() OVER (partition by v.stay_id order by r.charttime DESC) as lastRow
  from `{}` v 
  left join `physionet-data.mimic_icu.icustays` i on v.stay_id = i.stay_id 
  left join `physionet-data.mimic_derived.rhythm` r on i.subject_id = r.subject_id 
  where (r.charttime >= v.starttime ) and (r.charttime <= v.endtime )
) 
select stay_id , case when 
 (heart_rhythm = 'SR (Sinus Rhythm)' ) or
 (heart_rhythm = 'ST (Sinus Tachycardia)' ) or
 (heart_rhythm = 'SB (Sinus Bradycardia)' )
then 1 else 0 end as sinus_rhythm
from tmp
where lastRow=1
'''.format(view_id)
rhythm = run_query(sql_rhythm)

In [ ]:
sql_apsiii='''
select ap.*
FROM `{}` v
left join `physionet-data.mimic_derived.apsiii` ap on v.stay_id = ap.stay_id 
'''.format(view_id)
aps = run_query(sql_apsiii)

In [ ]:
sql_sofa='''
with vaso_stg as
(
  select ie.stay_id, 'norepinephrine' AS treatment, vaso_rate as rate
  FROM `physionet-data.mimic_icu.icustays` ie
  INNER JOIN `physionet-data.mimic_derived.norepinephrine` mv
    ON ie.stay_id = mv.stay_id
    AND mv.starttime >= DATETIME_SUB(ie.intime, INTERVAL '6' HOUR)
    AND mv.starttime <= DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
  UNION ALL
  select ie.stay_id, 'epinephrine' AS treatment, vaso_rate as rate
  FROM `physionet-data.mimic_icu.icustays` ie
  INNER JOIN `physionet-data.mimic_derived.epinephrine` mv
    ON ie.stay_id = mv.stay_id
    AND mv.starttime >= DATETIME_SUB(ie.intime, INTERVAL '6' HOUR)
    AND mv.starttime <= DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
  UNION ALL
  select ie.stay_id, 'dobutamine' AS treatment, vaso_rate as rate
  FROM `physionet-data.mimic_icu.icustays` ie
  INNER JOIN `physionet-data.mimic_derived.dobutamine` mv
    ON ie.stay_id = mv.stay_id
    AND mv.starttime >= DATETIME_SUB(ie.intime, INTERVAL '6' HOUR)
    AND mv.starttime <= DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
  UNION ALL
  select ie.stay_id, 'dopamine' AS treatment, vaso_rate as rate
  FROM `physionet-data.mimic_icu.icustays` ie
  INNER JOIN `physionet-data.mimic_derived.dopamine` mv
    ON ie.stay_id = mv.stay_id
    AND mv.starttime >= DATETIME_SUB(ie.intime, INTERVAL '6' HOUR)
    AND mv.starttime <= DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
)
, vaso_mv AS
(
    SELECT
    ie.stay_id
    , max(CASE WHEN treatment = 'norepinephrine' THEN rate ELSE NULL END) as rate_norepinephrine
    , max(CASE WHEN treatment = 'epinephrine' THEN rate ELSE NULL END) as rate_epinephrine
    , max(CASE WHEN treatment = 'dopamine' THEN rate ELSE NULL END) as rate_dopamine
    , max(CASE WHEN treatment = 'dobutamine' THEN rate ELSE NULL END) as rate_dobutamine
  from `physionet-data.mimic_icu.icustays` ie
  LEFT JOIN vaso_stg v
      ON ie.stay_id = v.stay_id
  GROUP BY ie.stay_id
)
, pafi1 as
(
  -- join blood gas to ventilation durations to determine if patient was vent
  select ie.stay_id, bg.charttime
  , bg.pao2fio2ratio
  , case when vd.stay_id is not null then 1 else 0 end as IsVent
  from `physionet-data.mimic_icu.icustays` ie
  LEFT JOIN `physionet-data.mimic_derived.bg` bg
      ON ie.subject_id = bg.subject_id
      AND bg.charttime >= DATETIME_SUB(ie.intime, INTERVAL '6' HOUR)
      AND bg.charttime <= DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
  LEFT JOIN `physionet-data.mimic_derived.ventilation` vd
    ON ie.stay_id = vd.stay_id
    AND bg.charttime >= vd.starttime
    AND bg.charttime <= vd.endtime
)
, pafi2 as
(
  -- because pafi has an interaction between vent/PaO2:FiO2, we need two columns for the score
  -- it can happen that the lowest unventilated PaO2/FiO2 is 68, but the lowest ventilated PaO2/FiO2 is 120
  -- in this case, the SOFA score is 3, *not* 4.
  select stay_id
  , min(case when IsVent = 0 then pao2fio2ratio else null end) as PaO2FiO2_novent_min
  , min(case when IsVent = 1 then pao2fio2ratio else null end) as PaO2FiO2_vent_min
  from pafi1
  group by stay_id
)
-- Aggregate the components for the score
, scorecomp as
(
select ie.stay_id
  , v.mbp_min
  , mv.rate_norepinephrine
  , mv.rate_epinephrine
  , mv.rate_dopamine
  , mv.rate_dobutamine

  , l.creatinine_max
  , l.bilirubin_total_max as bilirubin_max
  , l.platelets_min as platelet_min

  , pf.PaO2FiO2_novent_min
  , pf.PaO2FiO2_vent_min

  , uo.UrineOutput

  , gcs.gcs_min
from `physionet-data.mimic_icu.icustays` ie
left join vaso_mv mv
  on ie.stay_id = mv.stay_id
left join pafi2 pf
 on ie.stay_id = pf.stay_id
left join `physionet-data.mimic_derived.first_day_vitalsign` v
  on ie.stay_id = v.stay_id
left join `physionet-data.mimic_derived.first_day_lab` l
  on ie.stay_id = l.stay_id
left join `physionet-data.mimic_derived.first_day_urine_output` uo
  on ie.stay_id = uo.stay_id
left join `physionet-data.mimic_derived.first_day_gcs` gcs
  on ie.stay_id = gcs.stay_id
)
, scorecalc as
(
  -- Calculate the final score
  -- note that if the underlying data is missing, the component is null
  -- eventually these are treated as 0 (normal), but knowing when data is missing is useful for debugging
  select stay_id
  -- Respiration
  , case
      when PaO2FiO2_vent_min   < 100 then 4
      when PaO2FiO2_vent_min   < 200 then 3
      when PaO2FiO2_novent_min < 300 then 2
      when PaO2FiO2_novent_min < 400 then 1
      when coalesce(PaO2FiO2_vent_min, PaO2FiO2_novent_min) is null then null
      else 0
    end as respiration

  -- Coagulation
  , case
      when platelet_min < 20  then 4
      when platelet_min < 50  then 3
      when platelet_min < 100 then 2
      when platelet_min < 150 then 1
      when platelet_min is null then null
      else 0
    end as coagulation

  -- Liver
  , case
      -- Bilirubin checks in mg/dL
        when bilirubin_max >= 12.0 then 4
        when bilirubin_max >= 6.0  then 3
        when bilirubin_max >= 2.0  then 2
        when bilirubin_max >= 1.2  then 1
        when bilirubin_max is null then null
        else 0
      end as liver

  -- Cardiovascular
  , case
      when rate_dopamine > 15 or rate_epinephrine >  0.1 or rate_norepinephrine >  0.1 then 4
      when rate_dopamine >  5 or rate_epinephrine <= 0.1 or rate_norepinephrine <= 0.1 then 3
      when rate_dopamine >  0 or rate_dobutamine > 0 then 2
      when mbp_min < 70 then 1
      when coalesce(mbp_min, rate_dopamine, rate_dobutamine, rate_epinephrine, rate_norepinephrine) is null then null
      else 0
    end as cardiovascular

  -- Neurological failure (GCS)
  , case
      when (gcs_min >= 13 and gcs_min <= 14) then 1
      when (gcs_min >= 10 and gcs_min <= 12) then 2
      when (gcs_min >=  6 and gcs_min <=  9) then 3
      when  gcs_min <   6 then 4
      when  gcs_min is null then null
  else 0 end
    as cns

  -- Renal failure - high creatinine or low urine output
  , case
    when (creatinine_max >= 5.0) then 4
    when  UrineOutput < 200 then 4
    when (creatinine_max >= 3.5 and creatinine_max < 5.0) then 3
    when  UrineOutput < 500 then 3
    when (creatinine_max >= 2.0 and creatinine_max < 3.5) then 2
    when (creatinine_max >= 1.2 and creatinine_max < 2.0) then 1
    when coalesce(UrineOutput, creatinine_max) is null then null
  else 0 end
    as renal
  from scorecomp
)
select ie.subject_id, ie.hadm_id, ie.stay_id
  -- Combine all the scores to get SOFA
  -- Impute 0 if the score is missing
  , coalesce(respiration,0)
  + coalesce(coagulation,0)
  + coalesce(liver,0)
  + coalesce(cardiovascular,0)
  + coalesce(cns,0)
  + coalesce(renal,0)
  as SOFA
, respiration
, coagulation
, liver
, cardiovascular
, cns
, renal
from `physionet-data.mimic_icu.icustays` ie
left join scorecalc s
  on ie.stay_id = s.stay_id
where ie.stay_id in (select stay_id from `{}`)
'''.format(view_id)
sofa = run_query(sql_sofa)

In [ ]:
sql_comorbidities='''
WITH diag AS
(
    SELECT 
        hadm_id
        , CASE WHEN icd_version = 9 THEN icd_code ELSE NULL END AS icd9_code
        , CASE WHEN icd_version = 10 THEN icd_code ELSE NULL END AS icd10_code
    FROM `physionet-data.mimic_hosp.diagnoses_icd` diag
)
, com AS
(
    SELECT
        ad.hadm_id

        -- Myocardial infarction
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('410','412')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I21','I22')
            OR
            SUBSTR(icd10_code, 1, 4) = 'I252'
            THEN 1 
            ELSE 0 END) AS myocardial_infarct

        -- Congestive heart failure
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) = '428'
            OR
            SUBSTR(icd9_code, 1, 5) IN ('39891','40201','40211','40291','40401','40403',
                          '40411','40413','40491','40493')
            OR 
            SUBSTR(icd9_code, 1, 4) BETWEEN '4254' AND '4259'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I43','I50')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I099','I110','I130','I132','I255','I420',
                                                   'I425','I426','I427','I428','I429','P290')
            THEN 1 
            ELSE 0 END) AS congestive_heart_failure

        -- Peripheral vascular disease
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) IN ('440','441')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('0930','4373','4471','5571','5579','V434')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4431' AND '4439'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I70','I71')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I731','I738','I739','I771','I790',
                                                   'I792','K551','K558','K559','Z958','Z959')
            THEN 1 
            ELSE 0 END) AS peripheral_vascular_disease

        -- Cerebrovascular disease
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) BETWEEN '430' AND '438'
            OR
            SUBSTR(icd9_code, 1, 5) = '36234'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('G45','G46')
            OR 
            SUBSTR(icd10_code, 1, 3) BETWEEN 'I60' AND 'I69'
            OR
            SUBSTR(icd10_code, 1, 4) = 'H340'
            THEN 1 
            ELSE 0 END) AS cerebrovascular_disease

        -- Dementia
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) = '290'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('2941','3312')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('F00','F01','F02','F03','G30')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('F051','G311')
            THEN 1 
            ELSE 0 END) AS dementia

        -- Chronic pulmonary disease
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) BETWEEN '490' AND '505'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('4168','4169','5064','5081','5088')
            OR 
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J40' AND 'J47'
            OR 
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J60' AND 'J67'
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I278','I279','J684','J701','J703')
            THEN 1 
            ELSE 0 END) AS chronic_pulmonary_disease

        -- Rheumatic disease
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) = '725'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('4465','7100','7101','7102','7103',
                                                  '7104','7140','7141','7142','7148')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('M05','M06','M32','M33','M34')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('M315','M351','M353','M360')
            THEN 1 
            ELSE 0 END) AS rheumatic_disease

        -- Peptic ulcer disease
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) IN ('531','532','533','534')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('K25','K26','K27','K28')
            THEN 1 
            ELSE 0 END) AS peptic_ulcer_disease

        -- Mild liver disease
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) IN ('570','571')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('0706','0709','5733','5734','5738','5739','V427')
            OR
            SUBSTR(icd9_code, 1, 5) IN ('07022','07023','07032','07033','07044','07054')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('B18','K73','K74')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('K700','K701','K702','K703','K709','K713',
                                                   'K714','K715','K717','K760','K762',
                                                   'K763','K764','K768','K769','Z944')
            THEN 1 
            ELSE 0 END) AS mild_liver_disease

        -- Diabetes without chronic complication
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 4) IN ('2500','2501','2502','2503','2508','2509') 
            OR
            SUBSTR(icd10_code, 1, 4) IN ('E100','E10l','E106','E108','E109','E110','E111',
                                                   'E116','E118','E119','E120','E121','E126','E128',
                                                   'E129','E130','E131','E136','E138','E139','E140',
                                                   'E141','E146','E148','E149')
            THEN 1 
            ELSE 0 END) AS diabetes_without_cc

        -- Diabetes with chronic complication
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 4) IN ('2504','2505','2506','2507')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('E102','E103','E104','E105','E107','E112','E113',
                                                   'E114','E115','E117','E122','E123','E124','E125',
                                                   'E127','E132','E133','E134','E135','E137','E142',
                                                   'E143','E144','E145','E147')
            THEN 1 
            ELSE 0 END) AS diabetes_with_cc

        -- Hemiplegia or paraplegia
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) IN ('342','343')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('3341','3440','3441','3442',
                                                  '3443','3444','3445','3446','3449')
            OR 
            SUBSTR(icd10_code, 1, 3) IN ('G81','G82')
            OR 
            SUBSTR(icd10_code, 1, 4) IN ('G041','G114','G801','G802','G830',
                                                   'G831','G832','G833','G834','G839')
            THEN 1 
            ELSE 0 END) AS paraplegia

        -- Renal disease
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) IN ('582','585','586','V56')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('5880','V420','V451')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '5830' AND '5837'
            OR
            SUBSTR(icd9_code, 1, 5) IN ('40301','40311','40391','40402','40403','40412','40413','40492','40493')          
            OR
            SUBSTR(icd10_code, 1, 3) IN ('N18','N19')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I120','I131','N032','N033','N034',
                                                   'N035','N036','N037','N052','N053',
                                                   'N054','N055','N056','N057','N250',
                                                   'Z490','Z491','Z492','Z940','Z992')
            THEN 1 
            ELSE 0 END) AS renal_disease

        -- Any malignancy, including lymphoma and leukemia, except malignant neoplasm of skin
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) BETWEEN '140' AND '172'
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '1740' AND '1958'
            OR
            SUBSTR(icd9_code, 1, 3) BETWEEN '200' AND '208'
            OR
            SUBSTR(icd9_code, 1, 4) = '2386'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('C43','C88')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C00' AND 'C26'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C30' AND 'C34'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C37' AND 'C41'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C45' AND 'C58'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C60' AND 'C76'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C81' AND 'C85'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C90' AND 'C97'
            THEN 1 
            ELSE 0 END) AS malignant_cancer

        -- Moderate or severe liver disease
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 4) IN ('4560','4561','4562')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '5722' AND '5728'
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I850','I859','I864','I982','K704','K711',
                                                   'K721','K729','K765','K766','K767')
            THEN 1 
            ELSE 0 END) AS severe_liver_disease

        -- Metastatic solid tumor
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) IN ('196','197','198','199')
            OR 
            SUBSTR(icd10_code, 1, 3) IN ('C77','C78','C79','C80')
            THEN 1 
            ELSE 0 END) AS metastatic_solid_tumor

        -- AIDS/HIV
        , MAX(CASE WHEN 
            SUBSTR(icd9_code, 1, 3) IN ('042','043','044')
            OR 
            SUBSTR(icd10_code, 1, 3) IN ('B20','B21','B22','B24')
            THEN 1 
            ELSE 0 END) AS aids
    FROM `physionet-data.mimic_core.admissions` ad
    LEFT JOIN diag
    ON ad.hadm_id = diag.hadm_id
    GROUP BY ad.hadm_id
)

SELECT 

cohort.stay_id 
    , congestive_heart_failure
    , cerebrovascular_disease
    , dementia
    , chronic_pulmonary_disease
    , rheumatic_disease
    , mild_liver_disease
    , diabetes_without_cc
    , diabetes_with_cc
    , paraplegia
    , renal_disease
    , malignant_cancer
    , severe_liver_disease 
    , metastatic_solid_tumor 
    , aids


FROM `{}` cohort 
left join `physionet-data.mimic_icu.icustays` icu on icu.stay_id = cohort.stay_id
left join com on com.hadm_id = icu.hadm_id 
'''.format(view_id)
comorb = run_query(sql_comorbidities)

# Merge data

In [ ]:
# first check out subscores in apsiii and sofa; may remove them to have a simpler feature set
sofa.columns

In [ ]:
aps.columns

In [ ]:
aps_col_full = ['stay_id', 'apsiii', 'apsiii_prob', 'hr_score',
       'mbp_score', 'temp_score', 'resp_rate_score', 'pao2_aado2_score',
       'hematocrit_score', 'wbc_score', 'creatinine_score', 'uo_score',
       'bun_score', 'sodium_score', 'albumin_score', 'bilirubin_score',
       'glucose_score', 'acidbase_score', 'gcs_score']
sofa_col_full = ['stay_id', 'SOFA', 'respiration',
       'coagulation', 'liver', 'cardiovascular', 'cns', 'renal']

aps_col_short = ['stay_id', 'apsiii']
sofa_col_short= ['stay_id', 'SOFA']

In [ ]:
# merge all dataset to the cohort_df
ft_df = cohort_df.copy()

# first merge med1, med2 together and count vasos
med = med1.merge(med2)
med['count_of_vaso'] = med.epinephrine + med.vasopressin + med.dobutamine + med.norepinephrine + med.phenylephrine + med.dopamine

In [ ]:
for df in [demo, weight, height, bg, pf, vitals, med, vent, rrt, rhythm, neuroblock, comorb]:
    ft_df = ft_df.merge(df, how='left')
ft_df['duration'] /= 60.

In [ ]:
ft63 = ft_df.merge(sofa[sofa_col_short]).merge(aps[aps_col_short])
print('basic info', ft63.columns[:5])
print('features', ft63.columns[5:])
print('ft count', len(ft63.columns[5:]))

In [ ]:
ft_full = ft_df.merge(sofa[sofa_col_full]).merge(aps[aps_col_full])
print('basic info', ft_full.columns[:5])
print('features', ft_full.columns[5:])
print('ft count', len(ft_full.columns[5:]))

In [ ]:
# save it to csv
ft63.to_csv('ft63_invasive_procedureevents_based_cohort.csv')
# you can see it here
os.listdir('.')

In [ ]:
# download the file to your laptop
from google.colab import files
files.download('ft63_invasive_procedureevents_based_cohort.csv') 

### Etc

In [ ]:
# mortality extraction
sql_mortality = '''
SELECT c.* , a.subject_id ,a.hadm_id , a.admittime hosp_intime, a.dischtime hosp_outtime, i.intime icu_intime, i.outtime icu_outtime, a.deathtime 

FROM `{}` c
left join `physionet-data.mimic_icu.icustays` i on i.stay_id =c.stay_id 
left join `physionet-data.mimic_core.admissions` a on a.hadm_id =i.hadm_id 
order by stay_id
'''.format(view_id)
mort = run_query(sql_mortality)